# Nested Queries and Set Operations


In [6]:
import warnings
warnings.filterwarnings('ignore')
from sols4 import *

In [7]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%sql postgresql://osmaah02:@localhost/employees

'Connected: osmaah02@employees'

### 1.  How many employees have not worked for the development department?

In [9]:
%%sql

SELECT COUNT(emp_no) FROM 
    (select emp_no from dept_emp
        EXCEPT
    select emp_no from dept_emp natural join departments where dept_name = 'Development') AS NotDev

1 rows affected.


count
214317


### 1.a  Select the first and last names of the first 10 employees ordered by last name, first name that have not worked for the development department.

In [10]:
%%sql

SELECT first_name, last_name FROM
(
(SELECT emp_no FROM employees)

EXCEPT

(SELECT emp_no 
FROM departments 
NATURAL JOIN dept_emp WHERE dept_name = 'Development')) AS dno

NATURAL JOIN employees    
ORDER BY last_name, first_name
LIMIT 10;


10 rows affected.


first_name,last_name
Aemilian,Aamodt
Aleksander,Aamodt
Alexius,Aamodt
Alois,Aamodt
Amabile,Aamodt
Anestis,Aamodt
Anwar,Aamodt
Arlette,Aamodt
Arumugam,Aamodt
Arunachalam,Aamodt


### 2.  Give the first name, last name of the first 10 employees that have not been a Senior Engineer or Senior Manager, order by employee number.

In [11]:
#use Senior Staff or Senior Engineer for titles -> no such thing as senior manager

In [12]:
%%sql

SELECT first_name, last_name FROM
    (SELECT emp_no FROM titles
        EXCEPT
    SELECT emp_no FROM titles where title = 'Senior Staff' or title = 'Senior Engineer') As NotSenior 
    Natural Join employees
Order by emp_no
LIMIT 10

10 rows affected.


first_name,last_name
Bezalel,Simmel
Saniya,Kalloufi
Duangkaew,Piveteau
Mary,Sluis
Berni,Genin
Kazuhito,Cappelletti
Lillian,Haddadi
Mayuko,Warwick
Ramzi,Erde
Shahaf,Famili


### 3.  How many departments has each manager managed?

Your result should be in alphabetical order by last name, then first name

In [13]:
%%sql
SELECT first_name, last_name, COUNT
FROM
(SELECT emp_no, COUNT(*) 
 FROM dept_manager 
 GROUP BY emp_no) A -- deptmanager table has emp_no, dept_no, from_date, to_date

NATURAL JOIN employees
ORDER BY last_name, first_name;


24 rows affected.


first_name,last_name,count
Ebru,Alpin,1
Tonny,Butterworth,1
Rosine,Cools,1
Leon,DasSarma,1
Oscar,Ghazalie,1
Marjo,Giarratana,1
DeForest,Hagimont,1
Rutger,Hofmeyr,1
Przemyslawa,Kaelbling,1
Hilary,Kambil,1


### 4.  How many people were managed by Tonny Butterworth?

In [14]:
#The checker gives 205 for all those tony managed and left while he was a manger managed after

In [15]:
%%sql
SELECT COUNT(*) 
FROM
(SELECT dept_no, from_date, to_date
FROM  dept_manager -- Brad joined with 'departments NATURAL JOIN' -- don't think necessary' 
 NATURAL JOIN employees 
WHERE first_name = 'Tonny' and last_name = 'Butterworth') A

JOIN dept_emp ON
A.dept_no = dept_emp.dept_no 
AND 
((dept_emp.from_date<=A.from_date AND dept_emp.to_date>=A.to_date)
OR
(dept_emp.from_date>=A.from_date AND dept_emp.to_date<=A.to_date)
OR
(dept_emp.from_date>=A.from_date AND dept_emp.to_date>=A.to_date)
OR
(dept_emp.from_date<=A.from_date AND dept_emp.to_date<=A.to_date))
;

1 rows affected.


count
23580


### 5.  How many people have not been a manager?

In [16]:
%%sql

SELECT COUNT(*) FROM
(SELECT emp_no FROM employees
 
EXCEPT
 
SELECT emp_no FROM dept_manager) A;

1 rows affected.


count
300000


### 6.  What is the average salary for the 1980's?  What is the average salary for the 1990's?

Lets define this as salaries that both start and end in the same decade.  We will leave out any salaries that cross the boundary.

Format the salaries using '$999,999.99'

Your final answer should have two columns and two rows.  The columns should be labeled decade and average.

In [17]:
%%sql

SELECT '1980'AS decade, TO_CHAR(AVG(salary),'$999,999.99') average
FROM salaries
WHERE from_date >= '1980-01-01' AND to_date <= '1989-12-31'

UNION

SELECT '1990', TO_CHAR(AVG(salary), '$999,999.99')
FROM salaries
WHERE from_date >= '1970-01-01' AND to_date <= '1999-12-31'


;

2 rows affected.


decade,average
1980,"$ 54,985.51"
1990,"$ 61,296.23"


### 8.  How many people have had 10 or more raises since January 1, 1990

In [18]:
#The checker likes if you compare to the to-date

In [19]:
%%sql

SELECT count(*) FROM
(SELECT emp_no, COUNT(*) raisecount
FROM salaries
WHERE from_date >= '1979-05-05' --Quang suggests this is a more accurate date to compare with
GROUP BY emp_no) A
WHERE raisecount >= 5;

1 rows affected.


count
245432
